In [1]:
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import gutenberg as gb
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize 
import pandas as pd
#import lux

古腾堡语料库 NLTK包含古腾堡项目（Project Gutenberg）电子文档的一小部分文本。 该项目大约有25000（现在是36000）本免费电子书。 我们通过平均句子长度和平均词种数（词语丰富度）这两个特征，来看不同作者的写作风格。

NLTK的古腾堡语料库收集的都是不同作家的书

In [ ]:
#使用nltk的数据集
#nltk.download('gutenberg')
#nltk.download('punkt')
#nltk.download('movie_reviews')#电影评论
#nltk.download('webtext') #网络和聊天文本
from nltk.corpus import gutenberg
from nltk.corpus import webtext
from nltk.corpus import movie_reviews
#print(gutenberg.fileids())
send_table = [] # 得到text的二维str数组
text_string = '' # text纯string表示
p = ''',.";:'?()-{}=+/?()*&^%#;[]!`'''
data_source = movie_reviews
for text_id in data_source.fileids():
    print(text_id)
    unit = data_source.sents(text_id)
    for s in unit:
        sent = []
        for word in s:
            if len(word)==1 and word in p:
                continue #在此增加if条件可以去除停用词等
            word = word.lower()
            sent.append(word)
            text_string+=word+' '
        send_table.append(sent)

In [3]:
#使用torch上的数据集
# import datasets
import torch
import string
import re
from nltk.tokenize import sent_tokenize
from torchtext.datasets import IMDB
import torchtext2sendtable as t2s

train_iter = IMDB(split='train')
send_table = t2s.torchtext2nltkMLE(train_iter,30000)

In [4]:
#test
#send_table = []
#sent1 = ['i', 'am', 'sam']
#sent2 = ['sam' ,'i', 'am']
#sent3 = ['i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham']
#send_table.append(sent1)
#send_table.append(sent2)
#send_table.append(sent3)
text_string=''
for s in send_table:
    for word in s:
        word = word.lower()
        text_string+=word+' '

In [5]:
# 对text_string进行词频统计
word_list = []
cnt_list = []
text_tokenized = nltk.word_tokenize(text_string)
#print(text_tokenized)   
from nltk import FreqDist
fdist = FreqDist(text_tokenized)
print(fdist)
for a,b in fdist.items():
    word_list.append(str(a))
    cnt_list.append(b)
dic = {
    'word':word_list,
    'cnt':cnt_list
}
info_table = pd.DataFrame(dic)
info_table.head(10)

<FreqDist with 99526 samples and 5742728 outcomes>


,word,cnt
0,i,84586
1,rented,330
2,am,2653
3,curious-yellow,3
4,from,19636
5,my,11978
6,video,1512
7,store,485
8,because,8783
9,of,140952


In [ ]:
info_table.to_csv('./temp.csv')
df = pd.read_csv('./temp.csv')
from lux.vis.Vis import Vis
intent= ['word','cnt']
vis = Vis(intent,df)
vis
print(vis.to_matplotlib())

In [6]:
# Preprocess the tokenized text for 3-grams language modelling
# send_table: list[list(str)]
from nltk.lm.preprocessing import padded_everygram_pipeline
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, send_table)
# 使用nltk 的ngram模型
from nltk.lm import MLE
model = MLE(n) # Lets train a 3-grams model, previously we set n=3
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 118134 items>


In [7]:
model.score('am',['i'])

0.031682074225803845

In [8]:
model.logscore('cookie',['<s>'])

-inf

In [ ]:
# 2-gram
word2sum = {}
word2freq = {}
for sent in send_table:
    w1='<s>'
    for word in sent:
        freq = -model.logscore(word)
        log_prob = -model.logscore(word,[w1])
        #print(word+' :',end='')
        #print(log_prob)
        if word in word2sum:
            word2sum[word] += log_prob
            word2freq[word] += freq
        else:
            word2sum[word] = log_prob
            word2freq[word] = freq
        w1 = word

In [9]:
# 3-gram
word2sum = {}
word2freq = {}
for sent in send_table:
    w1=''
    w2=''
    for word in sent:
        freq = -model.logscore(word)
        if w2 == '':
            log_prob = -model.logscore(word,['<s>'])
        elif w1 == '':
            log_prob = -model.logscore(word,["<s>",w2])
        else:
            log_prob = -model.logscore(word,[w1,w2])
        if word in word2sum:
            word2sum[word] += log_prob
            word2freq[word] += freq
        else:
            word2sum[word] = log_prob
            word2freq[word] = freq
        w1 = w2
        w2 = word

In [10]:
#增加词长、计算信息量
word_count_table = pd.DataFrame()
for n,word in enumerate(info_table['word']):
    # Create a list of just the word we are interested in, we use regular expressions so that part of words do not count
    # e.g. 'ear' would be counted in each appearance of the word 'year'
    word_count = len(word)  
    word_count_table = word_count_table.append(pd.DataFrame({'len':word_count}, index=[n]))
info_table['len'] = word_count_table['len']
info_table.head()
sum_table = pd.DataFrame()
freq_table = pd.DataFrame()
for n,word in enumerate(info_table['word']):
    if word not in word2sum:
        sum_table = sum_table.append(pd.DataFrame({'sum':0}, index=[n]))
        freq_table = freq_table.append(pd.DataFrame({'freq':0},index=[n]))
        continue
    sum_proc = word2sum[word]
    freq = word2freq[word]
    sum_table = sum_table.append(pd.DataFrame({'sum':sum_proc}, index=[n]))
    freq_table = freq_table.append(pd.DataFrame({'freq':freq}, index=[n]))
info_table['sum_neg_log_prob'] = sum_table['sum']
info_table['sum_freq'] = freq_table['freq']
info_table['averge_content'] = (info_table['sum_neg_log_prob']/info_table['cnt'])
info_table['averge_freq'] = (info_table['sum_freq']/info_table['cnt'])
info_table.head(30)
info_table.to_csv('./info_table.csv')

In [ ]:
info_table.sort_values(by='cnt',ascending='',inplace=True)
info_table.dtypes
info_table = info_table.drop(info_table[info_table['word'].str.isdigit()==True].index)
info_table.intent = ['len','averge_content']
info_table.head(100)


In [ ]:
info_table = info_table[info_table['len']>4]
info_table = info_table.iloc[:20000]

In [ ]:
import matplot_section_cor as corshow
import numpy as np
corshow.show_cor_of(info_table,"len","averge_freq","averge_content","cnt")

In [ ]:
print (vis.to_matplotlib())

In [ ]:
#spearman correlation coefficient
#原始数据
X1= info_table['len']
Y1= info_table['averge_content']
Z1= info_table['averge_freq']
#s.corr()函数计算
r=X1.corr(Y1,method='spearman')
q=X1.corr(Z1,method='spearman')
print("corelationship of len and averge_content: %f",r)
print("corelationship of len and freq: %f",q)

In [ ]:
info_table.plot.scatter(x="len", y="averge_content",alpha=0.2)
plt.savefig('./3-gram_gutenberg_top_200.jpg', bbox_inches='tight', dpi=300)

In [ ]:
info_table_lenle3 = info_table[info_table['len']>5]
info_table_lenle3[info_table_lenle3['averge_content']<5].head(50)

In [ ]:
info_table.to_csv('./info_csv')
import lux
df = pd.read_csv('./info_csv')
from lux.vis.Vis import Vis
intent = ['averge_content',"len"]
vis = Vis(intent,df)
vis